## Problema 3
Realizamos la consulta requerida usando el paquete pgmpy.

In [1]:
# importación
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

Ahora declaramos el modelo para la detección de Ebola

In [60]:
#  Variables y dependencias del modelo
ebola_model = BayesianModel([('Ebola', 'Fiebre'),
                            ('Ebola', 'Sangrado'), 
                            ('Sangrado', 'Visita'),
                            ('Fiebre', 'Visita'),
                            ('Visita', 'Especialista'), 
                            ('Sangrado', 'Complicaciones')])

In [59]:
# Distribuciones condicionales
dpc_ebo = TabularCPD(variable='Ebola', variable_card=2, 
                    values=[[0.01],[0.99]])
dpc_fie = TabularCPD(variable='Fiebre', variable_card=2,
                    values=[[0.9, 0.4],
                            [0.1, 0.6]], 
                    evidence=['Ebola'],
                    evidence_card=[2])
dpc_san = TabularCPD(variable='Sangrado', variable_card=2,
                    values=[[0.95, 0.2],
                            [0.05, 0.8]], 
                    evidence=['Ebola'],
                    evidence_card=[2])
dpc_vis = TabularCPD(variable='Visita', variable_card=2,
                     values=[[1.0, 0.3, 0.5, 0.2],
                             [0.0, 0.7, 0.5, 0.8]],
                     evidence=['Fiebre', 'Sangrado'],
                     evidence_card=[2, 2])
dpc_com = TabularCPD(variable='Complicaciones', variable_card=2,
                    values=[[0.9, 0.25],
                            [0.1, 0.75]], 
                    evidence=['Sangrado'],
                    evidence_card=[2])
dpc_esp = TabularCPD(variable='Especialista', variable_card=2,
                    values=[[1.0, 0.2],
                            [0.0, 0.8]], 
                    evidence=['Visita'],
                    evidence_card=[2])

In [61]:
# Asociación de distribuciones a modelo
ebola_model.add_cpds(dpc_ebo, dpc_fie, dpc_san, dpc_vis, dpc_com, dpc_esp)

# verificación del modelo
ebola_model.check_model()

True

In [64]:
# instanciando objecto de eliminación de variables
ebola_inf = VariableElimination(ebola_model)

# consulta P(Ebola = falso | Especialista = 1)
consulta_ebo_esp = ebola_inf.query(variables=['Ebola'], evidence={'Especialista': 1})
#print(consulta_exp_ofe['Experiencia'])
print(consulta_ebo_esp)

+---------+--------------+
| Ebola   |   phi(Ebola) |
+=========+==============+
| Ebola_0 |       0.0013 |
+---------+--------------+
| Ebola_1 |       0.9987 |
+---------+--------------+


In [65]:
consulta_ebo_esp = ebola_inf.query(variables=['Visita'], evidence={'Complicaciones': 1})
print(consulta_ebo_esp)

+----------+---------------+
| Visita   |   phi(Visita) |
+==========+===============+
| Visita_0 |        0.2559 |
+----------+---------------+
| Visita_1 |        0.7441 |
+----------+---------------+
